幅広くデータを見ていく
* 加速度の波形を並べていみる
* ウィンドウフレームごとの平均，分散の変化

In [1]:
# 表示用
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook

In [2]:
import numpy as np
from numpy import zeros, newaxis
import datetime
import os
import copy
import time

# Parametor about Sensor
SensorNum=1 # Sensor number
#SensorAxis='AccX' # Axis

# Parametor about  " Sliding Window " 
WindowWidth =np.array([128,256,512,1024,2048,4096]) # Window Width
SlidingWidth =WindowWidth/4 #sliding window

# Parametor about Neural Network
AEDimention = 16

# Define Data Name
DATE= str( datetime.date.today() )+'/'
TITLE="Participants01_First_1/"

# Parametor about " Data Path"
DataPath = "/media/takeyama/Transfer/02_ActivityResearch/Paticipants01/01_First/20161018-105301/mem/"

#chopin
#StorePath = "/home/takeyama/Documents/"
#mozart
StorePath ="/media/takeyama/HD-PNFU3/01_ActivityResearchData/"
if not os.path.exists(StorePath+TITLE+'dictionary/'): 
    os.makedirs(StorePath+TITLE+'dictionary/')
DictionaryDataPath=StorePath+TITLE+'dictionary/'

if not os.path.exists(StorePath+TITLE+'window/'): 
    os.makedirs(StorePath+TITLE+'window/')
WindowDataPath=StorePath+TITLE+'window/'

if not os.path.exists(StorePath+TITLE+DATE+'studyOutput/'): 
    os.makedirs(StorePath+TITLE+DATE+'studyOutput/')
StudyOutputPath=StorePath+TITLE+DATE+'studyOutput/'

if not os.path.exists(StorePath+TITLE+DATE+'parametor/'): 
    os.makedirs(StorePath+TITLE+DATE+'parametor/')
ParametorPath=StorePath+TITLE+DATE+'parametor/'

if not os.path.exists(StorePath+TITLE+DATE+'result/'): 
    os.makedirs(StorePath+TITLE+DATE+'result/')
ResultPath=StorePath+TITLE+DATE+'result/'

if not os.path.exists(StorePath+'/'+TITLE+DATE+'graph/'): 
    os.makedirs(StorePath+'/'+TITLE+DATE+'graph/')
GraphPath=StorePath+'/'+TITLE+DATE+'graph/'

センサデータの読み込み

In [3]:
# センサデータのcsvファイル名をリストに集約する関数
def SensorDataFileNameList(path):
    l = os.listdir(DataPath)
    FilePath =[]
    SensorName = []
    
    for n in range(len(l)):
        if (l[n][:4].find('mem-') != -1):
            FilePath.append(l[n])
            SensorName.append(l[n][4:15])
    
    return FilePath,SensorName

CSVを読み込んで，CSVの中身を辞書型で表現する

In [4]:
# FFT library
from scipy import fftpack
from scipy import signal

# Graph Plot
import matplotlib.pyplot as plt
from matplotlib import gridspec

# Generate WindowFrame Function
from mymodule import window

# timer
import time

%matplotlib inline

In [5]:
DictName = os.listdir(DictionaryDataPath)
print DictName

['Chest.npz', 'LeftHand.npz', 'LeftLeg.npz', 'RightHand.npz', 'Rightleg.npz', 'West.npz']


In [6]:
Sensor5 = np.load(DictionaryDataPath+DictName[0])['data'][()]
Sensor1 = np.load(DictionaryDataPath+DictName[1])['data'][()]
Sensor3 = np.load(DictionaryDataPath+DictName[2])['data'][()]
Sensor2 = np.load(DictionaryDataPath+DictName[3])['data'][()]
Sensor4 = np.load(DictionaryDataPath+DictName[4])['data'][()]
Sensor6 = np.load(DictionaryDataPath+DictName[5])['data'][()]

In [7]:
def GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH):
    """
    data　ウィンドウフレームに変換するデータ
    registName　ウィンドウフレームに登録するデータの名前
    windowWidth　ウィンドウ幅
    slidingWidth　スライド幅
    PATH ウィドウフレームを保存するディレクトリ
    
    
    About Function:
        与えられたdataからウィンドウ幅windowWidth,スライド幅slidingWidthにしたがって
        registNameのウィドウフレームを返す．
        また，与えれたPATH内に同様なパラメータ( WindowWidth, slidingWidth)かつ同様な
        windowNameのものがある場合，そのデータを返す．
        この関数が登録，ウィンドウフレームに変換できるデータは１つとする
    """
    storedName = windowName+'_Win='+str(wWidth).zfill(4)+'_Sld='+str(sWidth).zfill(4)+'.npz'
    l = os.listdir(PATH)
    
    if storedName in l:
        print "this data had finished making"
        return np.load(PATH+storedName)['data'][()]
    
    w=window()
    w.SetData(windowName,dataRaw)    
    wind=w.Compile(wWidth,sWidth)
    windoW=wind.reshape((len(wind),np.prod(wind.shape[1:])))
    
    np.savez(PATH+storedName,data=wind)

    return wind

In [8]:
windowNum = 256
slidingNum = windowNum/32

wind=GetWindowFrame('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)

this data had finished making


In [9]:
wind.shape

(134702, 256)

# Fourier Transformation

## Energy

In [ ]:
def Vector_Energy(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    
    fftwind = np.fft.fft(wind)  # calcurate fast fourier Transfromation
    fftwind = fftwind[:,1:fftwind.shape[1]/2-1] # Orthogonal and Colossus removed
    
    f = lambda x: np.real(x)**2+np.imag(x)**2

    Output = np.array([])
    SqrF = np.array([])
    
    for i in tqdm_notebook( range( fftwind.shape[0] ) ):
        for l in range(fftwind.shape[1]):
            SqrF = np.append(SqrF,f(fftwind[i,l]))
            
        SqrF = SqrF/SqrF.size
        Output = np.append(Output, SqrF**0.5)
    return Output

In [ ]:
Vector_Energy('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)

this data had finished making


Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


In [ ]:
def Vector_Entropy(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    
    fftwind = np.fft.fft(wind)  # calcurate fast fourier Transfromation
    fftwind = fftwind[:,1:fftwind.shape[1]/2-1] # Orthogonal and Colossus removed
    
    f = lambda x: np.real(x)**2+np.imag(x)**2

    Output = np.array([])
    SqrF = np.array([])
    
    for i in tqdm_notebook( range( fftwind.shape[0] ) ):
        for l in range(fftwind.shape[1]):
            SqrF = np.append(SqrF,f(fftwind[i,l]))
            
        DistroF = fftwind[i,:]/np.sum(SqrF)

        for l in range(DistroF.size):
            tmp = np.append(tmp, DistroF[l]*np.log( DistroF[l] ) )
        Output = np.append(Output, -1*np.sum(tmp))
    return Output

In [ ]:
Vector_Entropy('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)

In [ ]:
def Vector_KLD(dataName, dataX, dataX, wWidth, sWidth, PATH):
    def f(p, q):
        """Calculates Kullback–Leibler divergence"""
        p = np.array(p)
        q = np.array(q)
        return np.sum(p * np.log(p / q), axis=(p.ndim - 1))
    
    for i in tqdm_notebook( range( dataX.shape[0] ) ):
        for l in range(dataY.shape[1]):
            Output = np.append(Output, f(dataX[i],dataY[l]))
    return Output

In [ ]:
def Vector_JSD(dataName, dataX, dataX, wWidth, sWidth, PATH):
    def f(p, q):
        """Calculates Jensen-Shannon Divergence"""
        p = np.array(p)
        q = np.array(q)
        m = 0.5 * (p + q)
        return 0.5 * kld(p, m) + 0.5 * kld(q, m)

    for i in tqdm_notebook( range( dataX.shape[0] ) ):
        for l in range(dataY.shape[1]):
            Output = np.append(Output, f(dataX[i],dataY[l]))
    return Output

In [ ]:
distX=Vector_Entropy('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)
distY=Vector_Entropy('AccY', Sensor1['AccY'], windowNum, slidingNum, WindowDataPath)

Vector_KLD('Sensor1',distX,distY, windowNum, slidingNum, WindowDataPath)

In [ ]:
Vector_JSD('Sensor1',distX,distY, windowNum, slidingNum, WindowDataPath)